In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

In [3]:
# split by time
df2 = pd.read_csv('ecommerce/olist_orders_dataset.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [12]:
df2['order_purchase_timestamp'].value_counts()

2018-06-01 13:39:44    3
2018-08-02 12:06:07    3
2018-08-02 12:05:26    3
2018-07-28 13:11:22    3
2018-02-19 15:37:47    3
                      ..
2018-07-03 22:19:03    1
2017-07-11 11:50:22    1
2018-07-17 15:03:18    1
2017-03-13 17:03:55    1
2018-08-06 22:04:22    1
Name: order_purchase_timestamp, Length: 98875, dtype: int64

In [13]:
df2['order_purchase_timestamp'].head()

0    2017-10-02 10:56:33
1    2018-07-24 20:41:37
2    2018-08-08 08:38:49
3    2017-11-18 19:28:06
4    2018-02-13 21:18:39
Name: order_purchase_timestamp, dtype: object

In [22]:
df2_purchase_time = df2.loc[:, ['order_purchase_timestamp']]
df2_purchase_time

,order_purchase_timestamp
0,2017-10-02 10:56:33
1,2018-07-24 20:41:37
2,2018-08-08 08:38:49
3,2017-11-18 19:28:06
4,2018-02-13 21:18:39
...,...
99436,2017-03-09 09:54:05
99437,2018-02-06 12:58:58
99438,2017-08-27 14:46:43
99439,2018-01-08 21:28:27


In [95]:
from datetime import datetime  # 将日期+时间转化成数字以进行split
time_series = []
for i in range(99441):
    time_string = str(df2_purchase_time.iloc[i])[28:47]      # 单独分出年-月-日-时-分-秒
    time_numerical = datetime.fromisoformat(time_string).timestamp()     # 转成数字
    time_series.append(time_numerical)      

In [64]:
#[28:47]可行，以下方法验证，slice出来的全部是年月日+时分秒
#for i in range(9090, 10040):
#    print(str(df2_purchase_time.iloc[i])[28:47])

In [99]:
time_series.sort()       # 升序排列

In [107]:
import statistics        # 中位数在此
med = statistics.median(time_series)
print(med)

1516287876.0


In [109]:
print("25th percentile of timeseries : ", np.percentile(time_series, 25))
print("50th percentile of timeseries : ", np.percentile(time_series, 50))
print("75th percentile of timeseries : ", np.percentile(time_series, 75))

25th percentile of timeseries :  1505198779.0
50th percentile of timeseries :  1516287876.0
75th percentile of timeseries :  1525419736.0


In [110]:
# now define train_set and test_set, set at 50th percentile

In [113]:
train_set = time_series[:len(time_series)//2]
test_set = time_series[len(time_series)//2:]

In [ ]:
#然后卡壳